### Import libraries

In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet, Title

output_notebook()

Loading BokehJS ...

### Goal

<img src="../images/original/fig_0932.png">

#### Colors 

In [2]:
GRAY1, GRAY2, GRAY3 = '#231F20', '#414040', '#555655'
GRAY4, GRAY5, GRAY6 = '#646369', '#76787B', '#828282'
GRAY7, GRAY8, GRAY9 = '#929497', '#A6A6A5', '#BFBEBE'
BLUE1, BLUE2, BLUE3, BLUE4 = '#174A7E', '#4A81BF', '#94B2D7', '#94AFC5'
RED1, RED2, RED3 = '#800000', '#C3514E', '#E6BAB7', 
GREEN1, GREEN2 = '#0C8040', '#9ABB59'
ORANGE1 = '#F79747'

#### Font 

In [3]:
FONT = 'Arial'

#### Preprocessing 

In [4]:
dataset = pd.read_csv('../data/data_fig_0930_31_32.csv')
dataset.shape

(5, 5)

In [5]:
dataset.head()

,Interest,Before,After,Formatted Before,Formatted After
0,Bored,11,12,11%,12%
1,Not great,5,6,5%,6%
2,OK,40,14,40%,14%
3,Kind of interested,25,30,25%,30%
4,Excited,19,38,19%,38%


In [6]:
dataset_h = dataset[['Interest', 'Before', 'After']].set_index('Interest').T.rename_axis('When').sort_values('When', ascending=True).reset_index()

In [7]:
dataset_h['bored_formatted'] = dataset_h['Bored'].astype(str) + '%'
dataset_h['notgreat_formatted'] = dataset_h['Not great'].astype(str) + '%'
dataset_h['ok_formatted'] = dataset_h['OK'].astype(str) + '%'
dataset_h['kind_formatted'] = dataset_h['Kind of interested'].astype(str) + '%'
dataset_h['excited_formatted'] = dataset_h['Excited'].astype(str) + '%'

In [8]:
dataset_h.sort_values('When', ascending=False, inplace=True)
dataset_h['When'] = dataset_h['When'].str.upper()

In [9]:
dataset_h.head()

Interest,When,Bored,Not great,OK,Kind of interested,Excited,bored_formatted,notgreat_formatted,ok_formatted,kind_formatted,excited_formatted
1,BEFORE,11,5,40,25,19,11%,5%,40%,25%,19%
0,AFTER,12,6,14,30,38,12%,6%,14%,30%,38%


In [10]:
# Bokeh doesn't allow you to add off-axis text. This is a little trick to do it
# This code adds extra space on the left
new_row = {'When':''}
dataset_h = dataset_h.append(new_row, ignore_index=True)

#### Plot

In [12]:
# Set the source of the plot
source = ColumnDataSource(dataset_h)

source_after = ColumnDataSource(dataset_h.loc[dataset_h['When'] == 'AFTER']) # Set source for after labels
source_before = ColumnDataSource(dataset_h.loc[dataset_h['When'] == 'BEFORE']) # Set source for before labels

# We going to iterate and then, adding lines for each category
categories = dataset_h.set_index('When').columns.tolist()[:5]
labels = dataset_h.set_index('When').columns.tolist()[5:]


# Set color by category
colors = [GRAY8, GRAY8, ORANGE1, BLUE2, BLUE2]


# Create the figure
p = figure(x_range=list(dataset_h['When']),
           y_range=(0, 45), 
           plot_height=500, 
           plot_width=830, 
           title='Pilot program was a success' + ' ' * 31, 
           toolbar_location='above')


# Add bars to the figure
for i, category in enumerate(categories):
    p.line(x='When', 
           y=category,           
           color=colors[i],
           line_width=5,
           source=source)

    p.circle(x='When', 
             y=category,                  
             color=colors[i],
             fill_color=colors[i],
             line_width=5,    
             size=10,
             source=source)
    
    
    # Add labels before
    p.add_layout(LabelSet(x='When', 
                          y=category, 
                          x_offset=-30,
                          y_offset=-10,
                          text=labels[i],
                          text_align='center',
                          text_baseline='bottom', 
                          text_color=colors[i], 
                          text_font=FONT,
                          text_font_size='13pt',
                          source=source_before))

    
    # Add labels after
    p.add_layout(LabelSet(x='When', 
                          y=category, 
                          x_offset=30,
                          y_offset=-10,
                          text=labels[i],
                          text_align='center',
                          text_baseline='bottom', 
                          text_color=colors[i], 
                          text_font=FONT,
                          text_font_size='13pt', 
                          source=source_after))


# Add subtitles
p.add_layout(Title(text="How do you feel about science?", 
                   align="left",
                   offset=85,
                   text_color=GRAY1,
                   text_font=FONT,
                   text_font_size='15pt',                   
                   text_font_style='normal'), 
             place="above")


# Add footing legend
p.add_layout(Title(text='Based on survey of 100 students conducted before and after pilot program (100% response rate on both surveys).', 
                   align="left",
                   offset=80,
                   text_color=GRAY6,
                   text_font=FONT,
                   text_font_size='10pt',
                   text_font_style='normal'), 
             "below")


# Text annotations
# This is a weakness of bokeh, you cannot format a single word within a text. We have to do it separately.

# Legends annotations
p.text(x=2, 
       y=38,
       x_offset=-75,
       y_offset=10,
       text=['Excited'],
       text_color=BLUE2,
       text_font=FONT,
       text_font_size='13pt')

p.text(x=2, 
       y=30,
       x_offset=-75,
       y_offset=35,
       text=['Kind of \ninterest'], 
       text_color=BLUE2,
       text_font=FONT,
       text_font_size='13pt')

p.text(x=2, 
       y=14,
       x_offset=-75,
       y_offset=10,
       text=['OK'], 
       text_color=ORANGE1,
       text_font=FONT,
       text_font_size='13pt')

p.text(x=2, 
       y=12,
       x_offset=-75,
       y_offset=11,
       text=['Bored'], 
       text_color=GRAY8,
       text_font=FONT,
       text_font_size='13pt')

p.text(x=2, 
       y=6,
       x_offset=-75,
       y_offset=8,
       text=['Not great'], 
       text_color=GRAY8,
       text_font=FONT,
       text_font_size='13pt')


# Before annotations
p.text(x=2, 
       y=33,
       x_offset=30,
       y_offset=-50,
       text=['BEFORE program, the'], 
       text_font=FONT,
       text_color=GRAY7,
       text_font_size='15pt',)

p.text(x=2, 
       y=35,
       x_offset=30,
       y_offset=20,
       text=['majority of children felt \njust Ok'],
       text_color=ORANGE1,
       text_font=FONT,
       text_font_size='15pt')

p.text(x=2, 
       y=35,
       x_offset=30,
       y_offset=20,
       text=['\t' * 13 + 'about science.'], 
       text_font=FONT,
       text_color=GRAY7,
       text_font_size='15pt')


# After annotations
p.text(x=2, 
       y=20,
       x_offset=30,
       y_offset=-40,
       text=['AFTER program,'], 
       text_color=GRAY7,
       text_font=FONT,
       text_font_size='15pt')

p.text(x=2, 
       y=20,
       x_offset=30,
       y_offset=40,
       text=['more children where \nKind of interested & \nExcited'],
       text_color=BLUE2,
       text_font=FONT,
       text_font_size='15pt')

p.text(x=2, 
       y=20,
       x_offset=40,
       y_offset=40,
       text=['\t' * 11 + 'about science.'], 
       text_color=GRAY7,
       text_font=FONT,
       text_font_size='15pt')
 
    
# I haven't found a way of styling de ticks. So, this is a little trick
p.segment(x0=0.5, 
          y0=0, 
          x1=1.5,
          y1=0, 
          color=GRAY4, 
          line_width=1)


# Elements attributes

# Modify title attributes
p.title.background_fill_color = GRAY7
p.title.offset = 85
p.title.text_color = 'white'
p.title.text_font = FONT
p.title.text_font_size = '24pt'
p.title.text_font_style = 'normal'


# Modify X axis attributes
p.xaxis.axis_line_color = None
p.xaxis.major_label_text_color = GRAY7
p.xaxis.major_label_text_font = FONT
p.xaxis.major_label_text_font_size = '12pt'
p.xaxis.major_label_standoff = 10
p.xaxis.major_tick_line_color = GRAY8
p.xaxis.major_tick_in = 0
p.xaxis.major_tick_out = 5
p.xaxis.minor_tick_line_color = None
p.xgrid.grid_line_color = None


# Modify Y axis attributes
p.yaxis.axis_line_color = None
p.yaxis.major_label_text_color = None
p.yaxis.major_tick_line_color = None
p.yaxis.minor_tick_line_color = None
p.ygrid.visible = False


# Handle backgrounds color
p.background_fill_color = 'white'
p.border_fill_color = 'white'
p.outline_line_color = 'white'


show(p)